# LightGBM

## 1. 개요

* Light Gradient Boosted Machine의 줄임말

* Ranking, classification 등 ML작업을 위해 이용되며 Decision tree algorithms에 기반을 두고 있음


## 2. 장점

* 빠른 학습 속도와 높은 효율성

* 적은 메모리 사용량

* 더 나은 정확도

* 병력 학습 지원

* large-scale data 취급 가능

## 3. 특징

** 정확도 최적화 **

* leaf-wise로 tree를 생성한다. - best first

* leaf를 생성하기위해 max delta loss인 leaf를 고른다.

* 고정된 leaf number를 가지면 leaf-wise 알고리즘이 level-wise 알고리즘에 비해 적은 손실을 이루는 경향이 있다.

* 다만 leaf-wise 방식은 data수가 적을때 과적합이 발생할 수 있으므로 LightGBM은 max-depth를 파라미터로 받아 나무의 깊이를 제한한다. 하지만 max_dept가 특정되어도 leaf-wise로 자란다.

* 불균형한 데이터를 다룰때 is_unbalance = false를 세팅한다.

https://github.com/microsoft/LightGBM/blob/master/docs/_static/images/level-wise.png

https://github.com/microsoft/LightGBM/blob/master/docs/_static/images/leaf-wise.png

cf. Best first (leaf-wise)

full tree를 생성하면 depth-first(level-wise)와 결과는 같다.

다만 나무가 뻗어나간 순서만 다르다. 다만 early stopping, pruning을 통해 나무를 fully grow 하지 않기에 이는 중요한 차이이다.

왜냐면 leaf-wise 방식은 특정 브랜치의 loss만을 따르지않고 분기마다 global loss의 기여에 따라 분기하기 때문에 이는 종종 lower-error trees를 level-wise보다 빨리 배운다.



** 범주형 자료 최적 split **

*  one-hot encoding은 tree learners에게 최선이 아님 - 특히 cardinality가 많을 수록 정확해지기 위해서 불균형하고 너무 깊은 트리가 발생함

* 대신 그것의 범주를 2개의 부분집합으로 나눈다. 만약 k개의 카테고리가 있다면 2^(k-1) - 1 의 부분으로 나뉜다. 시간복잡도는 O(k * log(k) )

* 자세히 말하자면 LightGBM은 범주형 자료를 그것의 합계값에 따라 히스토그램으로 정렬한다. 그리고 정렬된 히스그램에서 최고의 분기점을 찾는다.


** LightGBM 적용 **

* Binary classification를 하고 싶다면 logloss 사용 - Multi-classification은 multi_logloss 사용

* Regression을 하고 싶다면 L2loss 사용

** 주요 파라미터 (사이킷런 래퍼 기준)**

> n_estimators : 약한 학습기의 개수 (반복 수행 회수)

> learning_rate : 학습률. (0, 1)범위의 값 지정하며 부스팅 스텝을 반복적으로 수행할 때 업데이트 되는 학습률

> max_depth : 결정트리의 max_depth와 동일. 트리의 최대 깊이

> min_child_samples : 리프 노드가 될 수 있는 최소 데이터 건수 (sample 수)

> subsample : 트리가 커져서 과적합 되는 것을 제어하기 위해 데이터를 샘플링하는 비율 지정

> colsample_bytree : 트리 생성에 필요한 피쳐(컬럼)을 임의로 샘플링 하는 데 사용.

> reg_lambda (lambda_l2) : L2 규제 적용값. default = 1. 과적합 제어

> reg_alpha (lambda_l1) : L1 규제 적용값. default = 0. 과적합 제어

> early_stopping_rounds : 학습 조기 종료

> num_leaves : 최대 리프노드 갯수

> min_child_weight : 과적합 조절용. 결정트리의 min_child_leaf와 유사

## 4. 과적합을 해결하기 위해선

* path_smooth를 높인다

* 큰 training set를 사용한다

* lambda_l1, Lambda_l2, 그리고 min_gain_to_split으로 제약해본다.

* subsample값을 조정함

* 너무 깊은 나무를 만들지 않도록 한다
